<a href="https://colab.research.google.com/github/junsu122/AI_Basic/blob/main/CNN/alex_vgg_google_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 공통부분

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 1. 전처리 정의: 모든 모델은 224x224 크기를 표준으로 선호함
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 모델 입력 규격에 맞추기
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 데이터 수치 표준화
])

# 2. 데이터셋 다운로드 및 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"현재 사용 기기: {device}")

## AlexNet

In [ ]:
from torchvision import models
alexnet = models.alexnet(pretrained=True)
# 마지막 출력층을 우리 데이터셋 클래스 개수(CIFAR-10은 10개)에 맞춤
alexnet.classifier[6] = nn.Linear(4096, 10)

train_model(alexnet, "AlexNet")

# VGG19

In [ ]:
vgg19 = models.vgg19(pretrained=True)
vgg19.classifier[6] = nn.Linear(4096, 10)

train_model(vgg19, "VGG19")

## GoogleNet

In [ ]:
googlenet = models.googlenet(pretrained=True)
googlenet.fc = nn.Linear(1024, 10)

train_model(googlenet, "GoogLeNet")

## ResNet

In [ ]:
resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(2048, 10)

train_model(resnet50, "ResNet50")

# 공통부분

In [ ]:
import torch.nn as nn
import torch.optim as optim

def train_model(model, name):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    print(f"\n--- {name} 학습 시작 ---")
    model.train()

    # 예시로 1에폭(전체 데이터 1회 주행)만 실행
    for epoch in range(1):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if i % 100 == 99:
                print(f'[{i+1}] loss: {loss.item():.3f}')
    print(f"{name} 학습 완료")

# 🧠 주요 CNN 아키텍처 모델별 특징 정리

각 모델은 이미지 인식 분야의 발전을 이끈 핵심 알고리즘들로, 구조적 특징과 장단점이 뚜렷합니다.

---

## 1. AlexNet (2012) - 딥러닝 시대의 개막
* **특징**: 5개의 컨볼루션 레이어와 3개의 전결합 레이어로 구성되었습니다.
* **핵심 기술**:
    * **ReLU** 활성화 함수를 최초로 도입하여 학습 속도를 높였습니다.
    * **Dropout**을 사용하여 과적합(Overfitting)을 방지했습니다.
    * 두 개의 GPU를 병렬로 연결하여 연산했습니다.
* **장점**: 딥러닝이 기존 컴퓨터 비전 기술보다 압도적임을 증명한 기념비적 모델입니다.

---

## 2. VGG19 (2014) - 깊이와 단순함의 미학
* **특징**: 모든 컨볼루션 층에 아주 작은 **3x3 필터**만 사용했습니다.
* **핵심 기술**: 필터 크기를 줄이는 대신 층을 깊게 쌓아(VGG16은 16층, VGG19는 19층) 비선형성을 극대화했습니다.
* **장점**: 구조가 매우 단순하고 규칙적이어서 아키텍처 변형 및 이해가 쉽습니다.
* **단점**: 파라미터(가중치) 개수가 너무 많아 메모리 소모가 큽니다.



---

## 3. GoogLeNet / Inception v1 (2014) - 연산 효율의 극치
* **특징**: **Inception 모듈**이라는 독특한 구조를 사용하여 가로 방향으로 층을 넓게 펼쳤습니다.
* **핵심 기술**:
    * 1x1, 3x3, 5x5 컨볼루션을 병렬로 수행하여 다양한 크기의 특징을 한 번에 잡습니다.
    * **1x1 컨볼루션**을 사용하여 채널 수를 줄임으로써 연산량을 획기적으로 낮췄습니다.
* **장점**: VGG보다 훨씬 깊지만(22층), 파라미터 수는 훨씬 적어 효율적입니다.



---

## 4. ResNet (2015) - 층 쌓기의 한계를 넘다
* **특징**: 100층이 넘는 초거대 신경망 학습을 가능하게 했습니다.
* **핵심 기술**: **Skip Connection (Residual Learning)**을 도입했습니다. 입력값을 출력값에 직접 더해줌으로써 층이 깊어져도 데이터 정보가 사라지는 '기울기 소실(Vanishing Gradient)' 문제를 해결했습니다.
* **장점**: 현대 딥러닝 모델의 표준(Standard)으로 불릴 만큼 강력한 성능과 안정성을 가집니다.



---

## 📊 모델 한눈에 비교 요약

| 모델명 | 주요 키워드 | 구조적 핵심 | 추천 상황 |
| :--- | :--- | :--- | :--- |
| **AlexNet** | GPU 병렬화, ReLU | 8 Layers | 딥러닝 기초 학습 및 가벼운 분류 |
| **VGG19** | 3x3 필터, 단순함 | 19 Layers | 특징 추출(Feature Extraction) 베이스 |
| **GoogLeNet** | Inception, 1x1 Conv | 22 Layers | 연산 자원이 제한된 환경 |
| **ResNet** | Skip Connection | 50~152 Layers | 가장 범용적이고 높은 성능 필요 시 |